In [1]:
import zipfile
import os

def unzip_file(zip_path, extract_to):
    """Unzips a file to the specified directory."""
    os.makedirs(extract_to, exist_ok=True)  # Ensure the destination folder exists

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Extracted files to: {extract_to}")

# Example usage:
zip_path = "/content/Final_Sampled.zip"  # Replace with your ZIP file path
extract_to = "/content/"  # Replace with your extraction folder path

unzip_file(zip_path, extract_to)


Extracted files to: /content/


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import os

def csv_to_image(csv_file, image_size=(1164, 1164), channels=['MEAN', 'STDV', 'NPIXELS']):
    """Convert CSV data into a 2D grid (image-like format)"""
    df = pd.read_csv(csv_file)

    # Create empty image array (Height, Width, Channels)
    image = np.zeros((image_size[0], image_size[1], len(channels)))

    # Fill image with values
    for _, row in df.iterrows():
        x, y = int(row['X']), int(row['Y'])
        image[x, y, :] = [row[channel] for channel in channels]

    return image



In [2]:
# Define CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1164, 1164, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 1162, 1162, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 581, 581, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 579, 579, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 289, 289, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 287, 287, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 143, 143, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2617472)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │     335,036,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 335,129,921 (1.25 GB)

 Trainable params: 335,129,921 (1.25 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import os
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
control_sampled_folder="/content/control_sampled"
sle_sampled_folder="/content/aggregated_sle_sampled"

# Load images and labels
X, y = [], []

for file in os.listdir(control_sampled_folder):
    X.append(csv_to_image(os.path.join(control_sampled_folder, file)))
    y.append(0)  # Control class

for file in os.listdir(sle_sampled_folder):
    X.append(csv_to_image(os.path.join(sle_sampled_folder, file)))
    y.append(1)  # SLE class

X = np.array(X)
y = np.array(y)

from sklearn.utils import shuffle

X, y = shuffle(X, y, random_state=42)


# Train-test spli
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)

# Train the CNN
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=8)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 528ms/step - accuracy: 0.9905 - loss: 0.0577 - val_accuracy: 1.0000 - val_loss: 5.2855e-04
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 410ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 1.0000 - val_loss: 0.0237
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 414ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 0.0788
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step - accuracy: 1.0000 - loss: 5.7030e-05 - val_accuracy: 1.0000 - val_loss: 0.1268
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 427ms/step - accuracy: 1.0000 - loss: 3.3501e-05 - val_accuracy: 0.8000 - val_loss: 0.1612
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 414ms/step - accuracy: 1.0000 - loss: 1.2901e-05 - val_accuracy: 0.8000 - val_loss: 0.1838
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step - accuracy: 1.0000 - loss: 1.2168e-05 - val_accuracy: 0.8000 - val_loss: 0.1984
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 415ms/step - accuracy: 1.0000 - loss: 8.2675e-06 - val_accu

In [ ]:
model.save("cnn.keras")

In [7]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Predict on test set
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32").flatten()

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Evaluation Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")

# Optional: detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Control", "SLE"]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Evaluation Metrics:
Accuracy : 0.8000
Precision: 0.6667
Recall   : 1.0000
F1 Score : 0.8000

Classification Report:
              precision    recall  f1-score   support

     Control       1.00      0.67      0.80         3
         SLE       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5

